# Tarea 1. Hechos estilizados del ciclo económico mexicano
Basándose en el cuaderno donde documentábamos algunos hechos estilizados del ciclo económico de Estados Unidos desde 1947-1948, construya una contabilidad nacional del sector privado de la economía de México desde 1993, aunque México solo cuenta con datos del mercado laboral trimestral desde 2005. Consideren que el consumo de bienes duraderos es inversión, al igual que el saldo de la balanza comercial, y donde cada magnitud esté deflactada por su propio defactor. Recuerden que sólo deben considerar la formación de capital privado y el consumo privado.

Tengan en cuenta que el acervo de bienes duraderos genera una renta para los hogares de la economía que debe ser tenida en cuenta en su contabilidad nacional y que es probable que algunas de las variables no se encuentren ajustadas estacionalmente, por lo que tendrán que aprender a desestacionalizar usando filtros [**`ARIMA-X13`**](https://www.census.gov/data/software/x13as.html) en la librería [`statsmodels`](https://www.statsmodels.org/dev/generated/statsmodels.tsa.x13.x13_arima_analysis.html). Traten de explicar brevemente el origen de sus series y fíjense en si las cifras son trimestrales o se presentan anualizadas, como las de Estados Unidos.

En esta tarea necesito que:
1. Documenten gráficamente el comportamiento del PIB por habitante en México (quizás solo puedan hacerlo desde 2005 con datos trimestrales)
2. Documenten gráficamente las ratios de consumo e inversión respecto del PIB
3. Calculen la productividad total de los factores y grafíquenla junto con el PIB por habitante
4. Grafiquen conjuntamente el componente cíclico del PIB por habitante, la productividad total de los factores, el consumo, la inversión privada y la balanza comercial, reportando también sus desviaciones típicas
5. Encuentre la matriz de correlaciones de las variables que graficó en 4

Discuta y compare las características de largo y corto plazo de la economía Méxicana respecto a lo documentado para Estados Unidos. **Pueden trabajar en grupos de 4 y será el 25% del primer examen**.

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import requests as rq
import shapely as shp
import geopandas as geopd
import INEGIpy
from scipy.optimize import fsolve
from pandas.tseries.offsets import QuarterEnd
path = '/home'


### Datos de Cuentas Nacionales

Cargamos los datos de cuentas nacionales 


In [4]:
tempq = pd.read_excel(
    path+'nipaMEX.xlsx', 
    sheet_name='cantidades'
)

## les agregamos un índice
idx = pd.date_range(
    '1993-01-01', 
    periods=len(tempq), 
    freq='Q'
)
tempq = tempq.set_index(idx)


FileNotFoundError: [Errno 2] No such file or directory: '/home/MACROAVANZADA/nipaMEX.xlsx'

In [21]:
from INEGIpy import Indicadores

In [22]:
api_key = "e110e78a-2798-4ca8-a542-2a7a16998245"
indicadores = Indicadores(api_key)

In [23]:
ids_indicadores_cn = ['497170','497171','497172','497174','497165','497187','497193', '497215']

In [24]:
nombres_indicadores_cn = ['CP','GP','IP','X','M','CDN','CDI','IG']

In [25]:
datos_indicadores_cn = indicadores.obtener_df(ids_indicadores_cn, nombres = nombres_indicadores_cn,inicio = '1993', banco = 'BIE' )

In [26]:
datos_indicadores_cn

,CP,GP,IP,X,M,CDN,CDI,IG
fechas,,,,,,,,
1993-03-01,1.017863e+06,134056.527,324289.553,1.760586e+05,2.235323e+05,42068.034,10435.761,42882.309
1993-06-01,1.073726e+06,139849.018,305642.005,1.904766e+05,2.413931e+05,41203.295,11459.533,30448.524
1993-09-01,1.079554e+06,140634.871,323636.657,1.873464e+05,2.416065e+05,43932.060,10990.093,42481.256
1993-12-01,1.158797e+06,154256.085,347513.681,2.125172e+05,2.636348e+05,48995.831,14176.972,74857.838
1994-03-01,1.125996e+06,161709.764,375637.400,2.091871e+05,2.670901e+05,42104.861,14330.769,70528.942
...,...,...,...,...,...,...,...,...
2021-09-01,1.684258e+07,2962699.643,5356368.100,1.046007e+07,1.151614e+07,693200.802,766013.391,725123.210
2021-12-01,1.808558e+07,3095882.555,5541300.429,1.194643e+07,1.227231e+07,698256.800,999921.450,824781.137
2022-03-01,1.836359e+07,3168580.163,5778149.745,1.155176e+07,1.215946e+07,661983.105,920882.263,698410.096


In [27]:
inpc_indicador = ['628194']
nombre_inpc = ['INPC']

In [28]:
datos_inpc_cn = indicadores.obtener_df(inpc_indicador, nombres = nombre_inpc,inicio = '1993', banco = 'BIE')    
datos_inpc_cn.drop(index=datos_inpc_cn.tail(4).index, inplace = True)
datos_inpc_cn

,INPC
fechas,
1993-01-01,12.977320
1993-02-01,13.083345
1993-03-01,13.159594
1993-04-01,13.235480
1993-05-01,13.311137
...,...
2022-06-01,122.044000
2022-07-01,122.948000
2022-08-01,123.803000


Como el INPC esta mensual lo que hice fue hacer una media ponderada para cada trimestre pasarlo a valores
trimestrales 

In [29]:
datos_inpc_cn_trim = datos_inpc_cn.resample('Q').mean()
datos_inpc_cn_trim = datos_inpc_cn_trim.set_index(datos_indicadores_cn.index)
datos_inpc_cn_trim

ValueError: Length mismatch: Expected 120 rows, received array of length 119

Deflactamos los indicadores de contabilidad nacional 

In [14]:
def deflactar_serie(serie):
    return serie / datos_inpc_cn_trim['INPC'].loc[serie.index] * 100

In [15]:
datos_indicadores_cn_def = datos_indicadores_cn.apply(deflactar_serie, axis=0)
datos_indicadores_cn_def

KeyError: "None of [DatetimeIndex(['1993-03-01', '1993-06-01', '1993-09-01', '1993-12-01',\n               '1994-03-01', '1994-06-01', '1994-09-01', '1994-12-01',\n               '1995-03-01', '1995-06-01',\n               ...\n               '2020-06-01', '2020-09-01', '2020-12-01', '2021-03-01',\n               '2021-06-01', '2021-09-01', '2021-12-01', '2022-03-01',\n               '2022-06-01', '2022-09-01'],\n              dtype='datetime64[ns]', name='fechas', length=119, freq=None)] are in the [index]"

### Datos Laborales

In [16]:
ids_indicadores_labor = ['289242','446563','446565','446621','446575']

In [17]:
nombres_indicadores_labor = ['POB','POB15','EMP','EMP_G','EMP_CP']

In [18]:
datos_indicadores_labor = indicadores.obtener_df(ids_indicadores_labor, nombres = nombres_indicadores_labor,inicio = '1993', banco = 'BIE')    
datos_indicadores_labor.drop(index=datos_indicadores_labor.tail(4).index, inplace = True)
datos_indicadores_labor

,POB,POB15,EMP,EMP_G,EMP_CP
fechas,,,,,
2005-03-01,106701738.0,73074978.0,41441076.0,1900256.0,9788414.0
2005-06-01,106999770.0,73330809.0,41676868.0,1971727.0,9898730.0
2005-09-01,107306131.0,73650105.0,42334898.0,1999132.0,10126492.0
2005-12-01,107615497.0,74111373.0,42863703.0,2084749.0,10064021.0
2006-03-01,107928527.0,74290117.0,42740936.0,2062270.0,9921033.0
...,...,...,...,...,...
2020-12-01,126880395.0,96640120.0,53124071.0,2531914.0,12046629.0
2021-03-01,127203297.0,96940869.0,52973270.0,2550957.0,11951065.0
2021-06-01,127488991.0,97662760.0,55242748.0,2597172.0,12508970.0


## Unimos los dataframes

In [20]:
data = pd.merge(
    datos_indicadores_cn_def, 
    datos_indicadores_labor, 
    left_index = True, 
    right_index= True)
data

,CP,GP,IP,X,M,CDN,CDI,IG,POB,POB15,EMP,EMP_G,EMP_CP
fechas,,,,,,,,,,,,,
2005-03-01,1.055439e+07,1.642384e+06,3.255215e+06,3.912113e+06,4.091400e+06,406569.503094,351689.154610,637901.592424,106701738.0,73074978.0,41441076.0,1900256.0,9788414.0
2005-06-01,1.117340e+07,1.714216e+06,3.313810e+06,4.321850e+06,4.482751e+06,462556.137253,408994.375799,660580.084247,106999770.0,73330809.0,41676868.0,1971727.0,9898730.0
2005-09-01,1.122843e+07,1.678613e+06,3.327532e+06,4.211399e+06,4.449117e+06,483093.270369,368145.419086,712029.924769,107306131.0,73650105.0,42334898.0,1999132.0,10126492.0
2005-12-01,1.145601e+07,1.773886e+06,3.501632e+06,4.528027e+06,4.900966e+06,491019.874071,515388.659682,893361.061496,107615497.0,74111373.0,42863703.0,2084749.0,10064021.0
2006-03-01,1.112266e+07,1.777080e+06,3.480484e+06,4.395912e+06,4.442950e+06,443420.202005,399615.788376,692294.273263,107928527.0,74290117.0,42740936.0,2062270.0,9921033.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-01,1.456680e+07,2.746849e+06,4.302994e+06,9.687434e+06,8.715754e+06,719430.192627,745340.481063,673338.970514,126880395.0,96640120.0,53124071.0,2531914.0,12046629.0
2021-03-01,1.425733e+07,2.703034e+06,4.453187e+06,8.565925e+06,8.929864e+06,577328.910227,656417.884250,556210.864988,127203297.0,96940869.0,52973270.0,2550957.0,11951065.0
2021-06-01,1.478050e+07,2.773083e+06,4.450329e+06,9.338626e+06,9.345891e+06,641774.478641,668640.208277,528832.054012,127488991.0,97662760.0,55242748.0,2597172.0,12508970.0


In [19]:
for col in data.columns:
    if col != "fechas":
        res = sm.tsa.x13_arima_analysis(data[col], x12path='/home/x13as', prefer_x13=True, freq = 'Q', outlier=False, trading=False)
        data[col] = res.seasadj.values


NameError: name 'data' is not defined

In [287]:
data

,CP,GP,IP,X,M,CDN,CDI,IG,POB,POB15,EMP,EMP_G,EMP_CP
fechas,,,,,,,,,,,,,
2005-03-01,1.087396e+07,1.654426e+06,3.373889e+06,4.113566e+06,4.406041e+06,443676.298066,401434.622307,705308.170327,1.067012e+08,7.301970e+07,4.170360e+07,1.927229e+06,9.921032e+06
2005-06-01,1.103960e+07,1.679453e+06,3.331896e+06,4.211041e+06,4.422609e+06,458170.978239,435389.204427,716334.457787,1.070022e+08,7.339458e+07,4.170960e+07,1.973758e+06,9.861098e+06
2005-09-01,1.118551e+07,1.719054e+06,3.303486e+06,4.211117e+06,4.387321e+06,461593.272293,396300.819752,713221.644181,1.073056e+08,7.362070e+07,4.221979e+07,1.994303e+06,9.993969e+06
2005-12-01,1.130110e+07,1.754580e+06,3.384159e+06,4.427702e+06,4.696007e+06,478727.037873,411681.027739,760648.220400,1.076141e+08,7.413414e+07,4.267768e+07,2.059087e+06,1.010451e+07
2006-03-01,1.146160e+07,1.792613e+06,3.605464e+06,4.617546e+06,4.777824e+06,481779.427611,447658.545775,765644.760994,1.079280e+08,7.423308e+07,4.301550e+07,2.091598e+06,1.005304e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-01,1.404815e+07,2.719844e+06,4.185433e+06,9.089644e+06,8.426303e+06,653834.707830,623388.120274,580586.372872,1.268722e+08,9.662856e+07,5.221170e+07,2.826365e+06,1.159579e+07
2021-03-01,1.452214e+07,2.697056e+06,4.481371e+06,9.084000e+06,9.274914e+06,627514.287881,703433.630387,591192.974908,1.272058e+08,9.697502e+07,5.334469e+07,2.509953e+06,1.225324e+07
2021-06-01,1.504348e+07,2.731515e+06,4.603217e+06,9.358388e+06,9.552711e+06,660221.362247,714021.337123,596908.348913,1.274990e+08,9.766438e+07,5.570990e+07,2.377201e+06,1.274910e+07


### Definimos la economía 

In [293]:
# Definimos inversion como la suma de la inversion privada
# y el consumo duradero de los hogares
data['CD'] = data['CDN'] + data['CDI']
data['I'] = data['IP'] + data['CD'] + data['X'] - data['M']
data['C'] = data['CP'] - data['CD']

#PIB preliminar
data['Yp'] = data['I'] + data['C']